In [1]:
import numpy as np 
import pandas as pd 
import pickle 
import matplotlib.pyplot as plt 

In [2]:
obj = pd.read_pickle('data/update2_pickle.pickle')

# Helper Functions 

In [3]:
#find index 
name_array = np.array(obj[0])

def index_finder(csv_name):

    #find the index from the loaded object
    idx = np.where(name_array == csv_name)[0][0]
    
    return idx 

index_finder('BTCUSD.csv')

180

In [4]:
#pull predicted values, last 100 values 

all_predicted_values = obj[1]

def preds(csv_name):
    
    #pull index predicted values
    values = pd.DataFrame(all_predicted_values[index_finder(csv_name)], 
                                   columns = ['Open', 'High', 'Low', 'Close'])
    
    #opening values
    predicted_values = np.array(values.iloc[:,0])
    
    return predicted_values

In [5]:
def actual(csv_name):
    
    #get actual opening values 
    df = pd.read_csv('data/{}'.format(csv_name))
    actual_vals = np.array(df.iloc[:,1])
    
    return actual_vals

# Profit Calculations 

### Precomputation Stage

In [6]:
#running average, std deviations for previous k hours (opening price) 
def moving_averages(csv_name, k, cut_off):
    
    """
    For specified csv:
        - return the weighted averages
        - std deviations by window size k
        
    In order to avoid unncessary risk, you can specifya period (say hour 85) to cut off buying
    
        
    Output: Dataframe with Actual Values within the last 100 Hours of Trading
            Predicted Values
            Moving Averages for specified-k Day Lag 
            Std Deviations for the Moving Averages 
    """
    
    #load actual/predicted values
    actual_values = actual(csv_name)[-100 - k - 2:]
    predicted_values = preds(csv_name)
     
    #start calculations for the moving averages 
    moving_avs = []
    moving_stds = []
    
    for i,j in enumerate(predicted_values):
    
        #values needed
        block = actual_values[i:k+i]

        #compute mean, std append
        avg, std = np.average(block), np.std(block)
        moving_avs.append(avg)
        moving_stds.append(std)
    
    
    #recast to be the final 100 hours 
    actual_values = actual_values[k+2:]
    #print(len(actual_values))
    
    #For Buy Decision: Predicted - Moving Averages <0 and less than std of mean 
    buy_decision = []
    net_values = []
    
    #Calculate
    for i in range(100):
        
        #Differences between predictions, moving average of actual values 
        difference = predicted_values[i] - moving_avs[i]
        
        #if you buy based on the prediction, this is the difference between predicted and actual
        actual_worth = predicted_values[i] - actual_values[i]
        
        
        #ther predicted value is less than the standard deviation
        if (difference < 0) & (abs(difference) > moving_stds[i]) & (i <= cut_off):  
            buy_decision.append(1)
            net_values.append(actual_worth)
            
        else: 
            buy_decision.append(0)
            net_values.append(0)
    
    
    return pd.DataFrame({'Actual Values':actual_values, 
                         'Predicted Values':predicted_values,
                        'Moving Averages for {} Day Lag'.format(k):moving_avs, 
                        'Std Deviations':moving_stds, 
                        'Buy Coin?': buy_decision, 
                        'Difference': net_values})

In [7]:
def buy_sell(df):
    
    """
    Given weighted averages, standard deviations, choose stocks purchased at indices within
    the last 100 hours to sell off. 
    
    Sell decision is where difference between predicted value of coin is at least one standard 
    deviation above the price the coin was purchased at 
    
    Returns a dataframe of the trades made and their respective statistics, assets held after 
    """
    
    sell_time = []
    sold_array = []
    baseline = []
    
    buy_decisions = np.where(df.iloc[:,4] == 1)[0]
    
    
    #in subsequent time periods, we take sell greedily 
    #1st time the predicted value is above 1 std, we sell 
    
    for time in buy_decisions: 
        
        for i in range(time, df.shape[0]): 
            
            #predicted value one standard deviation above actual value 
            #predicted pattern at hour i 
            predicted_value = df.iloc[i, 1]

            #purchase price 
            buy_time_actual = df.iloc[time, 0]

            #running adjusted std dev
            std_dev_actual = df.iloc[i, 3]

            #measure the potential difference between prediced price and the purchase, 
            potential_difference = predicted_value - buy_time_actual

            # if the potential difference is greater than 1 std at current time, sell at open else hold
            if potential_difference > std_dev_actual: 

                #sell point
                sell_point = df.iloc[i,0]
                cash_flow = sell_point-buy_time_actual
                baseline.append(cash_flow)

                sold_array.append(time)
                sell_time.append(i)
                break
    
    #DF Trades  
    rev_df = pd.DataFrame({'Asset Purchased at time t':sold_array, 
                       'Sold at Time':sell_time, 'Net Revenue': baseline})
    
    profitable_trades_ratio = len(np.where(np.array(baseline) > 0)[0])/len(baseline)
    
    
    #Total Earnings From Trades
    total_earnings = np.sum(baseline)
    
    #Purchased Assets Still Held 
    assets_held = list(set(buy_decisions) - set(sold_array))
    
    #Total Value of Assets Held
    assets_held_value = np.sum([df.iloc[i,0] for i in assets_held])
    
    
    return [rev_df, profitable_trades_ratio, total_earnings, assets_held, assets_held_value]

In [8]:
profit = 0
held = 0

#Money made
for i in obj[0]:
    data = moving_averages(i,25,85)
    performance = buy_sell(data)[2]
    profit += performance
    
#Value of Assets Still Held 
for i in obj[0]:
    data = moving_averages(i,25,85)
    asset = buy_sell(data)[4]
    held += asset
    
print(profit)
print(held)

9495.835443870023
1002.6839400000002


In [9]:
# # Keeping Track of Expenditures 

# sell_time = []
# sold_array = []
# baseline = []

# for time in buy_decisions:
    
   
#     #in subsequent time periods, we take sell greedily 
#     #1st time the predicted value is above 1 std, we sell 
#     for i in range(time+1, df.shape[0]): 
        
#         #predicted pattern at hour i 
#         predicted_value = df.iloc[i, 1]
        
#         #purchase price 
#         buy_time_actual = df.iloc[time, 0]
        
#         #running adjusted std dev
#         std_dev_actual = df.iloc[i, 3]
        
#         #measure the potential difference between prediced price and the purchase, 
#         potential_difference = predicted_value - buy_time_actual
        
#         # if the potential difference is greater than 1 std at current time, sell at open else hold
#         if potential_difference > std_dev_actual: 
            
#             #sell point
#             sell_point = df.iloc[i,0]
#             cash_flow = sell_point-buy_time_actual
#             baseline.append(cash_flow)
            
#             sold_array.append(time)
#             sell_time.append(i)
#             break

# rev_df = pd.DataFrame({'Asset Purchased at time t':sold_array, 
#                        'Sold at Time':sell_time, 'Net Revenue': baseline})
# total_earnings = np.sum(baseline)

# assets_held = list(set(buy_decisions) - set(sold_array))

# assets_held_value = np.sum([df.iloc[i,0] for i in assets_held])
# assets_held_value